In [1]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from catboost import Pool
from numpy import loadtxt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from mlxtend.classifier import StackingCVClassifier
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier

In [2]:
train = pd.read_csv("train.csv")

In [3]:
train.head()

,id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,...,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,year,win_team
0,0,25524,30668,27293,37787,25565,30598,30749,30987,21164,...,3.75,6.50,1.44,3.75,8.0,1.44,3.75,7.0,2008,1
1,1,41671,18823,25526,46078,25587,181995,175646,177874,50189,...,3.40,4.75,0.00,0.00,0.0,0.00,0.00,0.0,2016,0
2,2,46704,193807,426878,185625,426873,79747,25560,39999,278341,...,3.12,3.30,0.00,0.00,0.0,0.00,0.00,0.0,2015,0
3,3,41301,166679,280083,33605,426174,181054,240478,39578,521204,...,3.90,5.75,0.00,0.00,0.0,0.00,0.00,0.0,2015,1
4,4,69650,32870,30890,30602,93484,29581,182223,78324,30352,...,3.10,3.70,0.00,0.00,0.0,0.00,0.00,0.0,2014,1


In [4]:
test = pd.read_csv("test.csv")

In [5]:
test.head()

,id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,year
0,0,37421,56846,33960,282676,47565,102622,205278,75342,99047,...,1.85,3.7,4.75,1.83,3.60,4.40,1.83,3.4,4.00,2012
1,1,150242,566158,157630,109649,40662,150119,278473,215226,207243,...,1.13,9.0,21.00,0.00,0.00,0.00,0.00,0.0,0.00,2015
2,2,37439,24131,33856,33849,33893,37631,38160,5243,38433,...,2.38,3.4,2.75,2.45,3.25,2.75,2.50,3.2,2.75,2010
3,3,67949,7683,40677,45413,181051,41659,25751,6771,38792,...,2.88,3.4,2.40,2.70,3.20,2.40,2.60,3.2,2.40,2010
4,4,41671,24493,24792,39440,30285,24464,25595,31499,27680,...,4.00,3.5,1.83,4.00,3.30,1.90,4.00,3.5,1.83,2010


In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8786 entries, 0 to 8785
Data columns (total 55 columns):
id                8786 non-null int64
home_player_1     8786 non-null int64
home_player_2     8786 non-null int64
home_player_3     8786 non-null int64
home_player_4     8786 non-null int64
home_player_5     8786 non-null int64
home_player_6     8786 non-null int64
home_player_7     8786 non-null int64
home_player_8     8786 non-null int64
home_player_9     8786 non-null int64
home_player_10    8786 non-null int64
home_player_11    8786 non-null int64
away_player_1     8786 non-null int64
away_player_2     8786 non-null int64
away_player_3     8786 non-null int64
away_player_4     8786 non-null int64
away_player_5     8786 non-null int64
away_player_6     8786 non-null int64
away_player_7     8786 non-null int64
away_player_8     8786 non-null int64
away_player_9     8786 non-null int64
away_player_10    8786 non-null int64
away_player_11    8786 non-null int64
B365H             8

In [15]:
players = pd.read_csv("players.csv")

,id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,...,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,year,win_team
0,0,25524,30668,27293,37787,25565,30598,30749,30987,21164,...,3.75,6.50,1.44,3.75,8.0,1.44,3.75,7.0,2008,1
1,1,41671,18823,25526,46078,25587,181995,175646,177874,50189,...,3.40,4.75,0.00,0.00,0.0,0.00,0.00,0.0,2016,0
2,2,46704,193807,426878,185625,426873,79747,25560,39999,278341,...,3.12,3.30,0.00,0.00,0.0,0.00,0.00,0.0,2015,0
3,3,41301,166679,280083,33605,426174,181054,240478,39578,521204,...,3.90,5.75,0.00,0.00,0.0,0.00,0.00,0.0,2015,1
4,4,69650,32870,30890,30602,93484,29581,182223,78324,30352,...,3.10,3.70,0.00,0.00,0.0,0.00,0.00,0.0,2014,1


In [35]:
train.year.value_counts()

2015    1245
2012    1198
2014    1176
2013    1139
2011    1101
2010    1053
2009     909
2016     620
2008     345
Name: year, dtype: int64

In [43]:
players['date'] = pd.to_datetime(players['date'])

In [45]:
players['year'] = players['date'].dt.year

In [48]:
players.year.value_counts()
players.drop("date",axis = 1)

,player_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,year
0,505942,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,2016
1,505942,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,2015
2,505942,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,...,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0,2015
3,505942,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,...,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,2015
4,505942,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,...,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,2007
5,155782,74.0,76.0,left,high,medium,80.0,53.0,58.0,71.0,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2016
6,155782,74.0,76.0,left,high,medium,80.0,53.0,58.0,71.0,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2016
7,155782,73.0,75.0,left,high,medium,79.0,52.0,57.0,70.0,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2016
8,155782,73.0,75.0,left,high,medium,79.0,51.0,57.0,70.0,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015
9,155782,73.0,75.0,left,high,medium,79.0,51.0,57.0,70.0,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015


In [49]:
players = players.dropna()

In [51]:
players.head(18397)

,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,year
0,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,44.0,71.0,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,2016
1,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,44.0,71.0,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,2015
2,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,44.0,71.0,...,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0,2015
3,505942,2015-03-20,61.0,65.0,right,medium,medium,48.0,43.0,70.0,...,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,2015
4,505942,2007-02-22,61.0,65.0,right,medium,medium,48.0,43.0,70.0,...,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,2007


In [29]:
from sklearn.preprocessing import LabelEncoder
train['win_team'] = train['win_team'].astype('bool')

Y = train['win_team']
#encoder = LabelEncoder()
#encoder.fit(Y)
#encoded_Y = encoder.transform(Y)
train = train.drop('win_team',axis = 1)
test_size = 0.33
seed = 0
X_train, X_test, y_train, y_test = train_test_split(train, Y, test_size=test_size, random_state=seed)

In [1034]:


model = XGBClassifier(silent=False, 
                      eta = 0.1,
                      max_depth = 2,
                      gamma=8,
                      subsample = 0.9,
                      colsample_bytree = 0.4,
                      objective='binary:logistic', 
                      learning_rate=0.01,  
                      n_estimators=500, 
                      reg_alpha = 0.3,
                     )
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, eta=0.1, gamma=8, learning_rate=0.01,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.3,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.9)

In [1035]:
preds_class = model.predict(X_test)
# Get predicted probabilities for each class

# Get predicted RawFormulaVal
accuracy = accuracy_score(y_test, preds_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))   

Accuracy: 69.55%


In [974]:
y_pred = model.predict_proba(test)[:,1]

y_pred = pd.Series(y_pred)
result = pd.DataFrame()
result["id"] = test["id"]
y_pred.count()

7190

In [975]:
result["win_team"] = y_pred


In [976]:
result.to_csv("result.csv",index = False)

In [977]:
from sklearn.ensemble import GradientBoostingClassifier

In [1072]:
model = GradientBoostingClassifier(loss = 'exponential',
                                   learning_rate = 0.01,
                                   n_estimators = 500,
                                   subsample=0.8,
                                   min_samples_split = 0.4,
                                   random_state = 10,
                                  )

In [1073]:
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='exponential', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=0.4,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              n_iter_no_change=None, presort='auto', random_state=10,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [1074]:
preds_class = model.predict(X_test)
# Get predicted probabilities for each class

# Get predicted RawFormulaVal
accuracy = accuracy_score(y_test, preds_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))   


Accuracy: 69.62%


In [1075]:
y_pred = model.predict_proba(test)[:,1]

y_pred = pd.Series(y_pred)
result = pd.DataFrame()
result["id"] = test["id"]
y_pred.count()

7190

In [1078]:
# Get some classifiers to evaluate
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
seed = 1075
np.random.seed(seed)
# Create classifiers
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = SVC()
rg = RidgeClassifier()
clf_array = [rf, et, knn, svc, rg]
for clf in clf_array:
    vanilla_scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf, 
       max_samples=0.4, max_features=10, random_state=seed)
    bagging_scores = cross_val_score(bagging_clf, X_train, y_train, cv=10, 
       n_jobs=-1)
    
    print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]"  
                       .format(clf.__class__.__name__, 
                       vanilla_scores.mean(), vanilla_scores.std()))
    print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n"
                       .format(clf.__class__.__name__, 
                        bagging_scores.mean(), bagging_scores.std()))

Mean of: 0.656, std: (+/-) 0.014 [RandomForestClassifier]
Mean of: 0.702, std: (+/-) 0.015 [Bagging RandomForestClassifier]

Mean of: 0.663, std: (+/-) 0.013 [ExtraTreesClassifier]
Mean of: 0.694, std: (+/-) 0.015 [Bagging ExtraTreesClassifier]

Mean of: 0.553, std: (+/-) 0.012 [KNeighborsClassifier]
Mean of: 0.595, std: (+/-) 0.012 [Bagging KNeighborsClassifier]

Mean of: 0.615, std: (+/-) 0.000 [SVC]
Mean of: 0.615, std: (+/-) 0.000 [Bagging SVC]

Mean of: 0.697, std: (+/-) 0.008 [RidgeClassifier]
Mean of: 0.701, std: (+/-) 0.014 [Bagging RidgeClassifier]



In [1081]:
bagging_clf.fit(X_train,y_train)


BaggingClassifier(base_estimator=RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001),
         bootstrap=True, bootstrap_features=False, max_features=10,
         max_samples=0.4, n_estimators=10, n_jobs=None, oob_score=False,
         random_state=1075, verbose=0, warm_start=False)

In [1082]:
preds_class =clf.predict(X_test)
# Get predicted probabilities for each class

# Get predicted RawFormulaVal
accuracy = accuracy_score(y_test, preds_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))   


Accuracy: 69.17%


In [1041]:
# Example of hard voting 
from sklearn.ensemble import VotingClassifier
clf = [rf, et, knn, svc, rg]
eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')
for clf, label in zip([rf, et, knn, svc, rg, eclf], ['Random Forest', 'Extra Trees', 'KNeighbors', 'SVC', 'Ridge Classifier', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.66 (+/- 0.02) [Random Forest]
Accuracy: 0.67 (+/- 0.02) [Extra Trees]
Accuracy: 0.55 (+/- 0.01) [KNeighbors]
Accuracy: 0.61 (+/- 0.00) [SVC]
Accuracy: 0.70 (+/- 0.01) [Ridge Classifier]
Accuracy: 0.69 (+/- 0.01) [Ensemble]


In [1046]:
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from mlxtend.classifier import EnsembleVoteClassifier
ada_boost = AdaBoostClassifier()
grad_boost = GradientBoostingClassifier()
xgb_boost = XGBClassifier()
boost_array = [ada_boost, grad_boost, xgb_boost]
eclf = EnsembleVoteClassifier(clfs=[ada_boost, grad_boost, xgb_boost], voting='hard')
labels = ['Ada Boost', 'Grad Boost', 'XG Boost', 'Ensemble']
for clf, label in zip([ada_boost, grad_boost, xgb_boost, eclf], labels):
    scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
    print("Mean: {0:.3f}, std: (+/-) {1:.3f} [{2}]".format(scores.mean(), scores.std(), label))

Mean: 0.698, std: (+/-) 0.015 [Ada Boost]
Mean: 0.700, std: (+/-) 0.015 [Grad Boost]
Mean: 0.702, std: (+/-) 0.018 [XG Boost]
Mean: 0.702, std: (+/-) 0.015 [Ensemble]


In [1047]:
eclf.fit(X_train,y_train)

EnsembleVoteClassifier(clfs=[AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_lea...,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)],
            refit=True, verbose=0, voting='hard', weights=None)

In [1048]:
preds_class = model.predict(X_test)
# Get predicted probabilities for each class

# Get predicted RawFormulaVal
accuracy = accuracy_score(y_test, preds_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))   

Accuracy: 69.55%


In [1208]:
et = ExtraTreesClassifier(n_estimators  = 20,criterion ='entropy', max_depth = 5,min_samples_split= 3,min_samples_leaf= 8,
                          max_leaf_nodes = 7,bootstrap = True,oob_score = True)
et.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=5, max_features='auto', max_leaf_nodes=7,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=8, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [1210]:
preds_class =et.predict(X_test)
# Get predicted probabilities for each class

# Get predicted RawFormulaVal
accuracy = accuracy_score(y_test, preds_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))   

Accuracy: 69.52%


In [1211]:
y_pred = et.predict_proba(test)[:,1]

y_pred = pd.Series(y_pred)
result = pd.DataFrame()
result["id"] = test["id"]
y_pred.count()

7190

In [1212]:
result["win_team"] = y_pred


In [1213]:
result.to_csv("result.csv",index = False)

In [1215]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np


# для встроенных картинок
%pylab inline
# чуть покрасивше картинки:
figsize(12, 9)

import warnings
warnings.filterwarnings("ignore")

#plt.rcParams['figure.figsize'] = 10, 7.5
#plt.rcParams['axes.grid'] = True
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams['font.family'] = 'Vernada' # Ubuntu

plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('font', weight='bold')
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)

# чтобы был русский шрифт
from matplotlib import rc
 
font = {'family': 'Vernada', #Droid Sans
        'weight': 'normal'}
rc('font', **font)

Populating the interactive namespace from numpy and matplotlib


In [1220]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(train, Y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [1221]:
from sklearn.metrics import roc_auc_score

def run_and_plot(clf, X, y, label):
    # plt.figure(figsize=(6, 5))
    
    a = clf.predict(X)
    
    xx, yy = np.meshgrid(np.linspace(-3, 3, 500), np.linspace(-3, 3, 500))

    
    print (label + ' AUC-ROC  = ' + str( roc_auc_score(y, a) ))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 7))
    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), Z.max(), 20), cmap=plt.cm.bwr, alpha=0.3) # plt.cm.Blues_r cmap=plt.cm.Blues_r)
    #a_ = plt.contour(xx, yy, Z, levels=[threshold], linewidths=1, colors='black')
    #plt.contourf(xx, yy, Z, levels=[threshold, Z.max()], colors='#CCDDFF')
    
    # не всё... :100
    plt.scatter(X[:300, 0], X[:300, 1], c=y[:300], s=20, alpha=1.0)
    plt.xlim([-3, 3])
    plt.ylim([-3, 3])
    # plt.axis('tight')
    plt.axis('off')
    plt.title(label)

In [1226]:
from sklearn.datasets import make_moons
from sklearn.datasets import make_classification




train_X, test_X, train_y, test_y = train_test_split(train, Y, train_size=0.7, random_state=1999)

plt.figure(figsize=(8, 7))
plt.scatter(train_X[:, 0], train_X[:, 1], c=train_y, s=20, alpha=0.5, label='train')
plt.scatter(test_X[:, 0], test_X[:, 1], c=test_y, s=5, alpha=1.0, label='test')
plt.xlabel("first feature")
plt.ylabel("second feature")
plt.legend()
plt.show()

TypeError: unhashable type: 'slice'

<Figure size 576x504 with 0 Axes>

In [1227]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

knn1 = KNeighborsRegressor(n_neighbors=3)
knn1.fit(X_train, y_train)
run_and_plot(knn1, X_test, test_y, '3NN')

knn2 = KNeighborsRegressor(n_neighbors=10)
knn2.fit(X_train, y_train)
run_and_plot(knn2, X_test, test_y, '10NN')


rg0 = Ridge(alpha=0.01)
rg0.fit(X_train, y_train)
run_and_plot(rg0, test_X, test_y, 'ridge-0.01')

rg1 = Ridge(alpha=1.1)
rg1.fit(X_train, y_train)
run_and_plot(rg1, test_X, test_y, 'ridge-1.1')

rg2 = Ridge(alpha=100.1)
rg2.fit(X_train, y_train)
run_and_plot(rg2, test_X, test_y, 'ridge-100.1')


rf1 = RandomForestRegressor(n_estimators=100, max_depth=1)
rf1.fit(X_train, y_train)
run_and_plot(rf1, test_X, test_y, 'rf-d1')

rf2 = RandomForestRegressor(n_estimators=100, max_depth=5)
rf2.fit(X_train, y_train)
run_and_plot(rf2, test_X, test_y, 'rf-d5')


gbm1 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, nthread=-1, objective='regression')    
gbm1.fit(X_train, y_train)
run_and_plot(gbm1, test_X, test_y, 'gbm-d2')

gbm2 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, nthread=-1, objective='regression')    
gbm2.fit(X_train, y_train)
run_and_plot(gbm2, test_X, test_y, 'gbm-d5')

3NN AUC-ROC  = 0.5074080127855872


ValueError: query data dimension must match training data dimension

In [ ]:
models = [knn1, knn2,rg1, rg2, rf1, rf2, gbm1, gbm2] # , rf3
ens_model = Ridge()
s1 = DjStacking(models, ens_model)
s1.fit(train_X, train_y)
run_and_plot(s1, test_X, test_y, '1-stacking')


s2 = DjStacking(models, ens_model)
s2.fit(train_X, train_y, p=-1)
run_and_plot(s1, test_X, test_y, '2-stacking')

In [ ]:
ens_model = Ridge(0.001)
s1 = DjStacking(models, ens_model)
a = 0
e = []
for t in range(10):
    s1.fit(train_X, train_y, p=0.4)
    a += s1.predict(test_X, train_y)
    
    auc = roc_auc_score(test_y, a)
    print (auc)
    e.append(auc)

In [ ]:
e4 = e

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(np.arange(1,11), 0.900175155867 + 0*np.arange(1,11), c='black', label='best base')
#plt.plot(np.arange(1,11), e1, label='p=0.1', lw=2)
#plt.plot(np.arange(1,11), e2, label='p=0.2', lw=2)
#plt.plot(np.arange(1,11), e3, label='p=0.3', lw=2)
plt.plot(np.arange(1,11), e4, label='p=0.4', lw=2)
plt.xlabel(u'сколько раз сделан блендинг (с разным разбиением)')
plt.ylabel('AUC ROC')
plt.legend()

In [ ]:
# models = [knn3, knn5, knn10, rg0, rg1, rg2, rf1, rf2, rf3]
ens_model = Ridge(0.001)

s1 = DjStacking(models, ens_model)
a = 0
e = []
for t in range(2, 11):
    s1.fit(train_X, train_y, p=-1, cv=t, err=0.00)
    a = s1.predict(test_X, train_y)
    auc = roc_auc_score(test_y, a)
    print (auc)
    e.append(auc)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(np.arange(2,11), 0.900175155867 + 0*np.arange(2,11), c='black', label='best base')
plt.plot(np.arange(2,11), e1, label='err=0.0', lw=2)
#plt.plot(np.arange(2,11), e3, label='err=0.01', lw=2)
plt.plot(np.arange(2,11), e2, label='err=0.03', lw=2)
# plt.plot(np.arange(2,11), e4, label='p=0.4', lw=2)
plt.xlabel(u'число фолдов')
plt.ylabel('AUC ROC')
plt.legend()

In [3]:
tmp = [('3NN', 0.83541103796),
       ('5NN', 0.854724379878),
       ('10NN', 0.874500610889),
       ('0-ridge', 0.883067503586),
       ('1-ridge', 0.883065725807),
       ('10-ridge', 0.88306217025),
       ('1-rf', 0.773247232554),
       ('3-rf', 0.878800390578),
       ('5-rf', 0.893922841743),
       ('blending', 0.893879508391),
       ('stacking', 0.89432173081)]


xticks = [x[0] for x in tmp]
aucs = [x[1] for x in tmp]

plt.figure(figsize=(12, 6))
plt.bar(np.arange(len(aucs)), aucs, color='#0000AA', label=u'алгоритмы')
plt.xticks(np.arange(len(aucs))+0.5, xticks, rotation=45)

plt.bar([9, 10], [0.891809729693, 0.890628395835], color='#AA0000', label=u'после удаления лучшего')


plt.xlim([0, 11])
plt.ylim([0.75, 0.9])
plt.ylabel('AUC ROC')
plt.legend(loc=3)

NameError: name 'plt' is not defined

In [30]:

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [31]:

seed = 7
numpy.random.seed(seed)


In [34]:

# baseline model
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(54, input_dim=54, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [35]:

estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
Use tf.cast instead.
Results: 54.57% (10.50%)


In [ ]:
estimator.fit(X_train,y_train)

In [ ]:
preds_class =estimator.predict(X_test)
# Get predicted probabilities for each class

# Get predicted RawFormulaVal
accuracy = accuracy_score(y_test, preds_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))   